In [6]:

import pandas as pd


eventos = pd.read_csv('logs_entrenamiento.csv')

notas = pd.read_csv('notas.csv')

column_names_df = notas.columns


column_names_eventos = eventos.columns

# Print the column names
print("Column names of 'df':", column_names_df)
print("Column names of 'eventos':", column_names_eventos)

# Rename a single column
notas.rename(columns={'Username': 'username'}, inplace=True)

notas = notas.replace('Not Attempted', 0)

print(notas)


Column names of 'df': Index(['Unnamed: 0', 'Username', 'Grade', 'Grade Scaled',
       'Quiz 1: Uso de modelos matemáticos en la economía',
       'Quiz 2: Motivación y Función de Producción',
       'Quiz 3: Preferencias y Toma de Decisiones',
       'Quiz 4: Salario y Efectos',
       'Quiz 5: Explicando Diferencias y Conclusión',
       'Quiz 6: Introducción y conceptos iniciales',
       'Quiz 7: Prediciendo el resultado de un juego',
       'Quiz 8: Evaluación de Resultados',
       'Quiz 9: Determinación de Asignaciones', 'Quiz 10: Desigualdad',
       'Quiz 11: Conceptos centrales', 'Quiz 12: Profundizando lo aprendido',
       'Quiz 13: Conceptos centrales', 'Quiz 14: Profundizando lo aprendido',
       'Quiz 15: Conceptos centrales', 'Quiz 16: Profundizando lo aprendido',
       'Quiz 17: Introducción',
       'Quiz 18: Oferta, demanda y equilibrio de mercado',
       'Quiz 19: Cambios en la oferta y la demanda',
       'Quiz 20: Excedentes y la mano invisible',
       'Quiz 2

In [7]:
#ACÁ SE TOMA INFO DE LOF_TRAIN PARA SUPLIR LA DATA DE NOTAS


# Convert the 'time_column' to a datetime object
eventos['time'] = pd.to_datetime(eventos['time'])

# Create new columns for month, day, and time
eventos['month'] = eventos['time'].dt.month
eventos['day'] = eventos['time'].dt.day
eventos['hora'] = eventos['time'].dt.strftime('%H:%M:%S')




#----------------------------------

# DURACION DE SESIONES

# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = eventos.groupby(['username', 'month', 'day'])['time'].agg(['min', 'max']).reset_index()

#
grouped['time_difference'] = (grouped['max'] - grouped['min']).dt.total_seconds() / 3600

# Merge the 'time_difference' column from the 'grouped' DataFrame into the 'eventos' DataFrame
eventos = eventos.merge(grouped[['username', 'month', 'day', 'time_difference']], on=['username', 'month', 'day'], how='left')



#----------------------------------

#ESTADISTICOS DE DUACION DE SESIONES

# Group the DataFrame by 'usuarioname' and calculate the average and standard deviation of 'time_difference'
grouped = eventos.groupby('username')['time_difference'].agg(['mean', 'std']).reset_index()

# Rename the columns for clarity
grouped.columns = ['username', 'average_time_difference', 'std_time_difference']

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'average_time_difference', 'std_time_difference']].drop_duplicates(), on='username', how='left')



#----------------------------------

#DURACION EN DÍAS DE SU USO DE EOL

# Extract the date part
eventos['date'] = eventos['time'].dt.date

# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = eventos.groupby(['username'])['date'].agg(['min', 'max']).reset_index()

# Calculate the time difference in hours
grouped['duracionEOL'] = (grouped['max'] - grouped['min']).dt.days

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'duracionEOL']].drop_duplicates(), on='username', how='left')




In [8]:

#----------------------------------

#FRECUENCIA DE SESIONES POR MES


distinct_months = eventos['month'].unique()

# Create a new dataframe to store the results
result_data = {'username': eventos['username']}
for month in distinct_months:
    result_data[month] = eventos['username'].map(
        eventos[eventos['month'] == month].groupby('username')['day'].nunique()
    )

# Convert the dictionary to a DataFrame
result_df = pd.DataFrame(result_data)

# Rename multiple columns
result_df.rename(columns={'8': 'Agosto', '9': 'Septiembre', '10': 'Noviembre'}, inplace=True)


result_df.fillna(0, inplace=True)

# Merge the 'result_df' with the 'df_notas' dataframe based on the 'username' column
notas = notas.merge(result_df, on='username', how='left')

# Print the updated 'eventos' DataFrame
print(notas)


        Unnamed: 0                                           username  Grade  \
0                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
1                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
2                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
3                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
4                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
...            ...                                                ...    ...   
851444         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
851445         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
851446         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
851447         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
851448         613  714d13c3517b88defa5763d78fc0c840a4bc1ad6092c02...   0.00   

        Grade Scaled Quiz 1: Uso de mod

In [ ]:
notas.to_csv('data_para_entrenar.csv')